In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# Pandas tqdm 설정
tqdm.pandas()

# ---------------------------------------------------------
# 1. 데이터 로드
# ---------------------------------------------------------
if not os.path.exists('base_data.csv'):
    print("❌ 'base_data.csv'가 없습니다. 먼저 eda.ipynb를 실행해주세요.")
else:
    print("📂 'base_data.csv' 로딩 중...")
    df = pd.read_csv('base_data.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    
    print(f"   -> 원본 크기: {df.shape}")
    print("🛠️ 본격적인 Feature Engineering 시작...")

    # ---------------------------------------------------------
    # 2. 날짜 정보 세분화 (Time Decomposition)
    # ---------------------------------------------------------
    print("   [1/4] 날짜 정보 분해 중 (Year, Month, Day, Weekday)...")
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    
    # 주말 여부 (토=5, 일=6)
    df['IsWeekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)

    # ---------------------------------------------------------
    # 3. 도메인 지식 기반 변수 (Season, RushHour)
    # ---------------------------------------------------------
    print("   [2/4] 도메인 변수 생성 중 (Season, RushHour)...")
    
    # 계절 변수
    def get_season(m):
        if 3 <= m <= 5: return 1 # 봄
        elif 6 <= m <= 8: return 2 # 여름
        elif 9 <= m <= 11: return 3 # 가을
        else: return 4 # 겨울
    
    df['Season'] = df['Month'].progress_apply(get_season)

    # 출퇴근 시간 (평일 7-9시, 17-20시)
    def is_rush_hour(row):
        if row['IsWeekend'] == 1: return 0
        if row['Hour'] in [7, 8, 9, 17, 18, 19, 20]: return 1
        return 0
    
    df['IsRushHour'] = df.progress_apply(is_rush_hour, axis=1)

    # ---------------------------------------------------------
    # 4. 상호작용 변수 (Interaction Features) - 여기가 핵심!
    # ---------------------------------------------------------
    print("   [3/4] 상호작용 변수 생성 중 (Temphumidity, TempWind, RainHumidity)...")
    
    # ① Temphumidity (기온 * 습도) -> 불쾌지수 대용
    if 'Temp' in df.columns and 'Humidity' in df.columns:
        df['Temphumidity'] = df['Temp'] * df['Humidity']

    # ② TempWind (기온 * 풍속) -> 체감온도 대용 (추위 강조)
    if 'Temp' in df.columns and 'Wind' in df.columns:
        df['TempWind'] = df['Temp'] * df['Wind']

    # ③ RainHumidity (강수량 * 습도) -> 악천후 지수 (비오고 꿉꿉함)
    if 'Rain' in df.columns and 'Humidity' in df.columns:
        df['RainHumidity'] = df['Rain'] * df['Humidity']

    # ④ (선택) 과학적 공식 기반 파생변수
    # 불쾌지수 (THI) 공식
    if 'Temp' in df.columns and 'Humidity' in df.columns:
        df['THI'] = 9/5 * df['Temp'] - 0.55 * (1 - df['Humidity']/100) * (9/5 * df['Temp'] - 26) + 32
        
    # 체감온도 공식
    if 'Temp' in df.columns and 'Wind' in df.columns:
        df['SensibleTemp'] = 13.12 + 0.6215 * df['Temp'] - 11.37 * (df['Wind']**0.16) + 0.3965 * df['Temp'] * (df['Wind']**0.16)

    # ---------------------------------------------------------
    # 5. 최종 데이터 정리 및 저장
    # ---------------------------------------------------------
    print("   [4/4] 데이터 정리 및 저장...")
    
    # 모델 학습에 방해되는 날짜(Date), 연도(Year) 컬럼 제거 여부 결정
    # Year는 추세 반영을 위해 남겨둘 수도 있지만, 시계열 분할이 아니면 과적합 위험이 있어 보통 뺍니다.
    # 여기서는 학습용이므로 Date만 제거하고 Year는 포함해보겠습니다 (장기 추세 반영).
    drop_cols = ['Date'] 
    df_final = df.drop(columns=[c for c in drop_cols if c in df.columns])
    
    # 결측치 최종 방어 (0으로)
    df_final.fillna(0, inplace=True)

    print(f"✅ Feature Engineering 완료!")
    print(f"   최종 데이터 크기: {df_final.shape}")
    print(f"   생성된 컬럼 목록: {df_final.columns.tolist()}")
    
    # 저장
    df_final.to_csv('final_train_data.csv', index=False)
    print("💾 'final_train_data.csv' 저장 완료 (Modeling 단계로 이동)")

📂 'base_data.csv' 로딩 중...
   -> 원본 크기: (72942, 9)
🛠️ 본격적인 Feature Engineering 시작...
   [1/4] 날짜 정보 분해 중 (Year, Month, Day, Weekday)...
   [2/4] 도메인 변수 생성 중 (Season, RushHour)...


100%|██████████| 72942/72942 [00:00<00:00, 136398.70it/s]


   [3/4] 상호작용 변수 생성 중 (Temphumidity, TempWind, RainHumidity)...
   [4/4] 데이터 정리 및 저장...
✅ Feature Engineering 완료!
   최종 데이터 크기: (72942, 20)
   생성된 컬럼 목록: ['Hour', 'Count', 'Temp', 'Rain', 'Wind', 'Humidity', 'Snow', 'Visibility', 'Year', 'Month', 'Day', 'DayOfWeek', 'IsWeekend', 'Season', 'IsRushHour', 'Temphumidity', 'TempWind', 'RainHumidity', 'THI', 'SensibleTemp']
💾 'final_train_data.csv' 저장 완료 (Modeling 단계로 이동)
